In [2]:
from z3 import *

import numpy as np
from itertools import combinations
from utils import *
import math
import time

In [3]:
int(1.93)

1

In [3]:
np.inf -1

inf

In [11]:
### NON INSERITO


def print_items_for_each_courier(couriers_items, items_predecessors, sz, cpt):
    # Creare un dizionario per mappare ogni corriere ai suoi pacchi
    couriers_dict = {}
    for i, j in couriers_items:
        if i not in couriers_dict:
            couriers_dict[i] = []
        couriers_dict[i].append(j)
    #print('cour_dict', couriers_dict)

    n=0
    for value in couriers_dict.values():
        n = n+len(value)
    #print(n)


    # Creare un altro dizionario per mappare ogni pacco al suo predecessore
    
    predecessors_dict = {i: j for j, i in items_predecessors}
    #print('pred_dict', predecessors_dict)
    
    # Per ogni corriere, seguire la catena di predecessori per ottenere l'ordine corretto dei pacchi
    for courier, items in couriers_dict.items():
        print(f"Corriere {courier}: ", end='')
        item = n + courier
        order = [item]
        while item in predecessors_dict and predecessors_dict[item] not in order:
            item = predecessors_dict[item]
            order.append(item)
        item = predecessors_dict[item]
        order.append(item)
        print(" -> ".join(map(str, order)), end='   ')
        print('Peso trasportato: ', end='')
        weight=0
        for i in couriers_dict[courier]:
            weight += sz[i]

        print(weight, '/', cpt[courier])

In [12]:
def at_least_one_seq(bool_vars):
    return Or(bool_vars)

def at_most_one_seq(bool_vars, name):
    constraints = []
    n = len(bool_vars)
    s = [Bool(f"s_{name}_{i}") for i in range(n - 1)]
    constraints.append(Or(Not(bool_vars[0]), s[0]))
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2])))
    for i in range(1, n - 1):
        constraints.append(Or(Not(bool_vars[i]), s[i]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1])))
        constraints.append(Or(Not(s[i-1]), s[i]))
    return And(constraints)

def exactly_one_seq(bool_vars, name):
    return And(at_least_one_seq(bool_vars), at_most_one_seq(bool_vars, name))

def at_most_k_seq(bool_vars, k, name):
    constraints = []
    n = len(bool_vars)
    s = [[Bool(f"s_{name}_{i}_{j}") for j in range(k)] for i in range(n - 1)]
    constraints.append(Or(Not(bool_vars[0]), s[0][0]))
    constraints += [Not(s[0][j]) for j in range(1, k)]
    for i in range(1, n-1):
        constraints.append(Or(Not(bool_vars[i]), s[i][0]))
        constraints.append(Or(Not(s[i-1][0]), s[i][0]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][k-1])))
        for j in range(1, k):
            constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][j-1]), s[i][j]))
            constraints.append(Or(Not(s[i-1][j]), s[i][j]))
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2][k-1])))   
    return And(constraints)

def at_least_k_seq(bool_vars, k, name):
    return at_most_k_seq([Not(var) for var in bool_vars], len(bool_vars)-k, name)

def exactly_k_seq(bool_vars, k, name):
    return And(at_most_k_seq(bool_vars, k, name+'1'), at_least_k_seq(bool_vars, k, name))

In [13]:
def objective_function(item_pred, cour_item, n, m, D):
    distances=[0 for _ in range(m)]

    # (i,j) in item_pred => j is predecessor of i
    for i, j in item_pred:

        if i>=n and j<n:
            c=i-n
            distances[c]+=D[j][n]
        elif j>=n and i<n:
            c=j-n
            distances[c]+=D[n][i]
        elif j<n and i<n:
            found=False
            c=0
            while not found:
                if (c,i) in cour_item:
                    distances[c]+=D[j][i]
                    found=True
                c+=1
    return max(distances)

In [14]:
instance_n=5 #from 1 to 21

if instance_n<10:
    file_name='inst0'+str(instance_n)+'.dat'
else:
    file_name='inst'+str(instance_n)+'.dat'

file = open('./Instances/'+file_name, 'r')

splitted_file = file.read().split('\n')

m = int(splitted_file[0])
n = int(splitted_file[1])
cpt_tmp = list(map(int, splitted_file[2].split(' ')))
tmp_sz=splitted_file[3].split(' ')
if '' in tmp_sz:
    sz_tmp=list(map(int, [tmp_sz[i] for i in range(len(tmp_sz)) if tmp_sz[i]!='']))
else:
    sz_tmp = list(map(int, splitted_file[3].split(' ')))
D = [list(map(int, line.strip().split(' '))) for line in splitted_file[4:(n+5)]]

print('Instance number '+str(instance_n)+': '+str(n)+' items and '+str(m)+' couriers.')

Instance number 5: 3 items and 2 couriers.


In [7]:
def mcp(n, m, sz, cpt):
   '''
   Return a solver that represent the Multiple Courier Planning problem.

   input:   - n : int, number of items
            - m : int, number of couriers
            - sz: list of ints, array of item's sizes (len(sz) has to be equal to n)
            - cpt: array of couriers' capacities (len(cpt) has to be equal to m)
   output:  - solv: Solver, the solver with all the correct constraints
            - pred: list of lists of BoolRef, the incidence matrix vairable
            - c: list of lists of BoolRef, the courier-items matrix
   '''

   solv= Solver()
   max_w = np.max(cpt)

   #Create a m x (n+m) x max_w matrix c: c[i][j][0]==True iff the i-th courier takes the j-th item (or starting/ending point).
   #The third dimension is useful for the capacity constraint.
   c=[[[Bool(f"c({i})_{j}_[{w}]") for w  in range(max_w+1)] for j in range(n+m)]for i in range(m)]

   #Each pack have to be carried by exactly one courrier
   for j in range(n):
      solv.add(exactly_one_seq([c[i][j][0] for i in range(m)], f"one_c_for_item({j})"))

   #The last m coulmns of c are fixed: the courier cour starts and ends at the deposit n+cour
   for j in range(m):
      for i in range(m):
         solv.add(Not(Or(c[i][n+j][1:])))
         if j==i:
            solv.add(c[i][n+j][0])
         else:
            solv.add(Not(c[i][n+j][0]))


   for cour in range(m):
      for item in range(n):
         solv.add(Implies(c[cour][item][0], exactly_k_seq(c[cour][item][1:], sz[item], 'weight('+str(cour)+'-'+str(item)+')')))
         solv.add(Implies( Not(c[cour][item][0]), Not(Or(c[cour][item][1:])) ) )
   
   for cour in range(m):
      solv.add(at_most_k_seq([c[cour][item][k] for k in list(range(max_w+1))[1:] for item in range(n)], cpt[cour], 'capacity'+str(cour)))
   
   #Create a (n+m)x(n+m) matrix: pred[i][j] = true if the j-th item is the predecessor of the i-th item. 
   #The n+c column in the matrix is the starting point of the c-th courier. The n+c row in the matrix is the ending point of the c-th courier
   pred = [[Bool(f"pred({i})_{j}")for j in range(n+m)]for i in range(n+m)]

   #Each item/ending point has exactly one predecessor and each item/starting point is predecessor of exactly one other item. 
   for i in range(n+m):
      col_i = []
      for j in range(n+m):
         col_i += [pred[j][i]]

      solv.add(exactly_one_seq(col_i, f"PC_{i}"))
      solv.add(exactly_one_seq(pred[i], f"PR_{i}"))

   #If the courier cour has the item i and the item j is the predecessor of the item i then cour has the item j
   for cour in range(m):
      solv.add(And(  [Implies(And([ c[cour][i][0], pred[i][j]] ) , c[cour][j][0]) for i in range(n+m) for j in range(n+m)]  ))
   
   #Create a list of length n no_loop: if the item j is predecessor of the item i then no_loop(i) as an integer is > than no_loop(j) as an integer 

   no_loops = [[Bool(f"no_loops({item})_{j}")for j in range(n//(m-1)+1)]for item in range(n)]

   for item in range(n):
      for k in range(n//(m-1)+1):
         solv.add(Implies(Not(no_loops[item][k]), Not(Or(no_loops[item][:k])))) #k=False then all the above are false --> if j<k is true then all the below are true
   
   for item_i in range(n):
      for item_j in range(n):
         solv.add(Implies(pred[item_i][item_j], Or([And(no_loops[item_i][k], Not(no_loops[item_j][k]))for k in range(n//(m-1)+1)])  ))
   
   return solv, pred, c

In [8]:
cpt=[]
sz=[]

if np.max(cpt_tmp)>100:
    for i in range(m):
        cpt.append(cpt_tmp[i]//10)
    for i in range(n):
        sz.append(sz_tmp[i]//10+1)

else:
    cpt=cpt_tmp
    sz=sz_tmp

## If n is less than 35:

In [16]:
def mcp_tmp(n,m,sz,cpt):
    solv= Solver()
    max_w = np.max(cpt)

    #Create a m x (n+m) matrix c: c[i][j] ==True iff the i-th courier takes the j-th item (or starting/ending point).
    c=[[Bool(f"c({i})_{j})") for j in range(n+m)]for i in range(m)]

    #Each pack have to be carried by exactly one courrier
    for j in range(n):
        solv.add(exactly_one_seq([c[i][j] for i in range(m)], f"one_c_for_item({j})"))

    #The last m coulmns of c are fixed: the courier cour starts and ends at the deposit n+cour
    for j in range(m):
        for i in range(m):
            if j==i:
                solv.add(c[i][n+j])
            else:
                solv.add(Not(c[i][n+j]))
    
    #Weight constraint
    for cour in range(m):
        cour_weight = [c[cour][item] for item in range(n) for _ in range(sz[item])]
        solv.add(at_most_k_seq(cour_weight, cpt[cour], f"weight({cour})"))
   
    #Create a (n+m)x(n+m) matrix: pred[i][j] = true if the j-th item is the predecessor of the i-th item. 
    #The n+c column in the matrix is the starting point of the c-th courier. The n+c row in the matrix is the ending point of the c-th courier
    pred = [[Bool(f"pred({i})_{j}")for j in range(n+m)]for i in range(n+m)]

    #Each item/ending point has exactly one predecessor and each item/starting point is predecessor of exactly one other item. 
    for i in range(n+m):
        col_i = []
        for j in range(n+m):
            col_i += [pred[j][i]]

        solv.add(exactly_one_seq(col_i, f"PC_{i}"))
        solv.add(exactly_one_seq(pred[i], f"PR_{i}"))

    #If the courier cour has the item i and the item j is the predecessor of the item i then cour has the item j
    for cour in range(m):
        solv.add(And(  [Implies(And([ c[cour][i], pred[i][j]] ) , c[cour][j]) for i in range(n+m) for j in range(n+m)]  ))
   
    #Create a list of length n no_loop: if the item j is predecessor of the item i then no_loop(i) as an integer is > than no_loop(j) as an integer 
    nl_max = n//(m-1)+1
    no_loops = [[Bool(f"no_loops({item})_{j}")for j in range(nl_max)]for item in range(n)]

    for item in range(n):
        for k in range(n//(m-1)+1):
            solv.add(Implies(Not(no_loops[item][k]), Not(Or(no_loops[item][:k])))) #k=False then all the above are false --> if j<k is true then all the below are true
    #Example: no_loops[j] = [1 1 1 1 1 1 1 0 0 0]

    # If item_j is predecessor of item_j then no_loops[j]<no_loops[i] as integers    
    for item_i in range(n):
        for item_j in range(n):
            solv.add(Implies(pred[item_i][item_j], Or([  And(no_loops[item_i][k], Not(no_loops[item_j][k])) for k in range(nl_max)  ])  ))
    
    return solv, pred, c
   

In [15]:
starting_time = time.time()
timeout = starting_time + 60*5
check_timeout = timeout-time.time()

print('Start...')

solv, pred, c = mcp(n, m, sz, cpt)

print('Solver built in', 60*5 - (timeout-time.time()), 'seconds')

best_obj = np.mean([np.mean(i) for i in D])  *(n/m) +(np.mean( [np.mean(D[n][:-1]), np.mean([D[i][n] for i in range(n)]) ] )  * m)

while solv.check()==sat and time.time() < timeout:
    tmp_model = solv.model()
    
    item_pred, cour_item = [(i,j) for j in range(n+m) for i in range(n+m) if tmp_model.evaluate(pred[i][j])], [(i, j) for j in range(n) for i in range(m) if tmp_model.evaluate(c[i][j][0])] #tmp_model.evaluate(c[i][j][0])
    tmp_obj = objective_function(item_pred, cour_item, n, m, D)
    if tmp_obj<best_obj:
        best_solution=tmp_model
        best_obj=tmp_obj
        print(best_obj)

    check_timeout = timeout-time.time() #time left
    solv.set('timeout', int(check_timeout*1000)) #time left in millisec 
    solv.add(Not( And([pred[i][j] for i in range(n+m) for j in range(n+m) if tmp_model.evaluate(pred[i][j])])))
print_items_for_each_courier(cour_item, item_pred, sz, cpt)

Start...
Solver built in 4.054271936416626 seconds
206


## If n is more than 35

## Search for the best solution:

In [8]:
def objective_function_small(item_pred, n, D):
    distance=0
    for i, j in item_pred:
        if i>=n:
            distance+=D[j][n]
        elif j>=n:
            distance+=D[n][i]
        else:
            distance+=D[j][i]
    return distance


In [9]:
def mcp_small(n):

   #create Solver
   solv= Solver()
   
   '''
      pred matrix  
   '''
   #pred[i][j] = true if the j-th item is the predecessor of the i-th item. 
   #The n+i column in the matrix is the starting point of the i-th courier. The n+i row in the matrix is the ending point of the i-th courier
   pred = [[Bool(f"pred({i})_{j}")for j in range(n+1)]for i in range(n+1)]

   #Each item/ending point has exactly one predecessor and each item/starting point is predecessor of exactly one other item. 
   #i=0 to i=n+m-1
   #j=0 to j=n+m-1 
   for i in range(n+1):
      col_i = []
      for j in range(n+1):
         col_i += [pred[j][i]]

      solv.add(exactly_one_seq(col_i, f"PC_{i}"))
      solv.add(exactly_one_seq(pred[i], f"PR_{i}"))
   
   no_loops = [[Bool(f"no_loops({item})_{j}")for j in range(n)]for item in range(n)]

   for item in range(n):
      for k in range(n):
         solv.add(Implies(Not(no_loops[item][k]), Not(Or(no_loops[item][:k])))) #k=False then all the above are false --> if j<k is true then all the below are true
   
   for item_i in range(n):
      for item_j in range(n):
         solv.add(Implies(pred[item_i][item_j], Or([And(no_loops[item_i][k], Not(no_loops[item_j][k]))for k in range(n)])  ))
   
   return solv, pred


In [10]:
def clustering(D):
    list_of_mins=[]
    n=len(D)-1
    Closest=[]
    for item in range(n): 
        tmp = copy.deepcopy(D[item])
        tmp[item]=1000
        Closest.append(np.argmin(tmp[:-1])) 
        list_of_mins.append(np.min(tmp[:-1]))
    #print(list_of_mins)
    clusters=[]
    already_in_cluster=[]
    for item in range(n):
        if item==Closest[Closest[item]] and (item not in already_in_cluster):
            clusters.append([item, Closest[item]])
            already_in_cluster.append(item)
            already_in_cluster.append(Closest[item])
        elif item!=Closest[Closest[item]]:
            clusters.append([item])
    clusters.append([n])
    D_new=[]
    n_new=len(clusters)-1

    for row in range(n_new): 
        D_new.append([]) 
        for column in range(n_new): 
            if row==column:
                D_new[row].append(0)
            else:
                tmp_dist=[]
                for i in clusters[row]:
                    for j in clusters[column]:
                        tmp_dist.append(D[i][j])
                D_new[row].append(np.mean(tmp_dist))
        tmp_dist=[]
        for i in clusters[row]:
            tmp_dist.append(D[i][n])
        D_new[row].append(np.mean(tmp_dist))
    D_new.append([])

    for column in range(n_new):
        tmp_dist=[]
        for i in clusters[column]:
            tmp_dist.append(D[n][i])
        D_new[n_new].append(np.mean(tmp_dist))
    D_new[n_new].append(0)

    return D_new, clusters

In [11]:
starting_time = time.time()
timeout = starting_time + 60*5
check_timeout = timeout-time.time()

if n<60:
    clustering_iterations = 2
elif n<100:
    clustering_iterations = 3
elif n<130:
    clustering_iterations = 4
else:
    clustering_iterations = 5

D_opt=D
clusters=[ [i] for i in range(n+1) ]
for i in range(clustering_iterations):
    D_opt, new_clusters=clustering(D_opt)
    old_clusters=clusters
    clusters=[]
    for clus in new_clusters:
        clusters.append([])
        for item in clus:
            for old_item in old_clusters[item]:
                clusters[-1].append(old_item)

In [12]:
clusters_paths=[]
cluster_path_distances=[]
real_clusters = [cluster for cluster in clusters if len(cluster)>1]

for it in range(len(real_clusters)):
    cluster = real_clusters[it]
    print('working on' , cluster)
    n_cluster = len(cluster)
    
    timeout_for_clustering = starting_time + 60*1*((it+1)/len(real_clusters))

    print('time for this cluster', timeout_for_clustering-time.time())

    D_clus=[]
    for i in cluster:
        D_clus.append([])
        for j in cluster:
            D_clus[-1].append(D[i][j])
        D_clus[-1].append(0)
    D_clus.append([0 for k in range(n_cluster+1)])

    solv, pred = mcp_small(n_cluster)

    best_obj = 1000 #np.mean([np.mean(i) for i in D_clus])  *(n_cluster) +(np.mean( [np.mean(D_clus[n_cluster][:-1]), np.mean([D_clus[i][n_cluster] for i in range(n_cluster)]) ] ) )

    while solv.check()==sat and time.time() < timeout_for_clustering:

        tmp_model = solv.model()
    
        item_pred = [(i,j) for j in range(n_cluster+1) for i in range(n_cluster+1) if tmp_model.evaluate(pred[i][j])]
        tmp_obj = objective_function_small(item_pred, n_cluster, D_clus)
        if tmp_obj<best_obj:
            best_model=tmp_model
            best_obj=tmp_obj
        check_timeout = timeout_for_clustering-time.time() #time left
        solv.set('timeout', int(check_timeout*1000)) #time left in millisec 

        solv.add(Not( And([pred[i][j] for i in range(n_cluster+1) for j in range(n_cluster+1) if tmp_model.evaluate(pred[i][j])])))
       
    cluster_copy=copy.deepcopy(cluster)
    cluster_copy.append(-1)
    clusters_paths.append([(cluster_copy[i],cluster_copy[j]) for i in range(n_cluster+1) for j in range(n_cluster+1) if best_model.evaluate(pred[i][j])])
    cluster_path_distances.append(best_obj)
print(clusters_paths)

working on [0, 13, 27]
time for this cluster 4.235216856002808
working on [1, 29]
time for this cluster 8.466907739639282
working on [2, 5, 26]
time for this cluster 12.70992922782898
working on [4, 7, 28]
time for this cluster 16.93691110610962
working on [6, 36]
time for this cluster 21.163489818572998
working on [8, 34]
time for this cluster 25.415622234344482
working on [9, 44, 10]
time for this cluster 29.659219980239868
working on [11, 23, 46]
time for this cluster 33.88540530204773
working on [14, 24, 22]
time for this cluster 38.114452838897705
working on [17, 25]
time for this cluster 42.338468074798584
working on [19, 32]
time for this cluster 46.58961462974548
working on [30, 31, 43]
time for this cluster 50.84224033355713
working on [38, 42]
time for this cluster 55.077494382858276
working on [41, 45]
time for this cluster 59.33346915245056
[[(0, -1), (13, 0), (27, 13), (-1, 27)], [(1, -1), (29, 1), (-1, 29)], [(2, 26), (5, 2), (26, -1), (-1, 5)], [(4, -1), (7, 4), (28, 7),

In [13]:
n_new = len(clusters)-1

first_items_for_clusters=[]
last_item_for_clusters=[]

i=0
for clus in clusters:
    if len(clus)>1:
        path=clusters_paths[i]
        for couple in path:
            if couple[0]==-1:
                last_item_for_clusters.append(couple[1])
            elif couple[1]==-1:
                first_items_for_clusters.append(couple[0])
        i+=1
    else:
        last_item_for_clusters.append(clus[0])
        first_items_for_clusters.append(clus[0])

D_new=[]
for i in range(n_new+1):
    D_new.append([])
    for j in range(n_new+1):
        if j==i:
            D_new[-1].append(0)
        else:
            D_new[-1].append( D[first_items_for_clusters[i]][last_item_for_clusters[j]] )


sz_new=[]
for item in range(n_new):
    sz_new.append(sum([sz[i] for i in clusters[item]]))

print(cpt)

[30, 20, 20]


In [14]:
print('Start...')

solv, pred, c = mcp(n_new, m, sz_new, cpt)

print('built!')

best_obj = np.mean([np.mean(i) for i in D_new])  *(n_new/m) +(np.mean( [np.mean(D_new[n_new][:-1]), np.mean([D_new[i][n_new] for i in range(n_new)]) ] )  * m)

while solv.check()==sat and time.time() < timeout:
    tmp_model = solv.model()
    print(111)
    item_pred, cour_item = [(i,j) for j in range(n_new+m) for i in range(n_new+m) if tmp_model.evaluate(pred[i][j])], [(i, j) for j in range(n_new) for i in range(m) if tmp_model.evaluate(c[i][j][0])]
    tmp_obj = objective_function(item_pred, cour_item, n_new, m, D_new)
    if tmp_obj<best_obj:
        best_solution=tmp_model
        best_obj=tmp_obj
        print(best_obj)

    check_timeout = timeout-time.time() #time left
    solv.set('timeout', int(check_timeout*1000)) #time left in millisec 
    solv.add(Not( And([pred[i][j] for i in range(n_new+m) for j in range(n_new+m) if tmp_model.evaluate(pred[i][j])])))
print_items_for_each_courier(cour_item, item_pred, sz, cpt)

Start...


built!
